In [ ]:
# phase 1

import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import json
!pip install cloudscraper
import cloudscraper

# Create a CloudScraper instance
scraper = cloudscraper.create_scraper()

response = scraper.get('https://rentberry.com/apartments/s/buffalo-ny')
soup = BeautifulSoup(response.text, 'html.parser')
for property_info in soup.find_all('body', class_ = 'no-min-width'):
    # print(property_info)
    info = soup.find('script', text=re.compile(r'window\.SAVED_STORE_STATE\s*=\s*{.*}'))
    cleaned_js_object_str = info.string.strip()[len('window.SAVED_STORE_STATE = '):]

closing_brace_index = cleaned_js_object_str.rfind('}')
cleaned_js_object_str = cleaned_js_object_str[:closing_brace_index + 1]
json_object = json.loads(cleaned_js_object_str)
data = pd.DataFrame(json_object["search"]["data"]["properties"]["apartments"])

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import json
import cloudscraper
# Create a CloudScraper instance
scraper = cloudscraper.create_scraper()

response = scraper.get('https://rentberry.com/apartments/s/buffalo-ny')
soup = BeautifulSoup(response.text, 'html.parser')
for property_info in soup.find_all('body', class_ = 'no-min-width'):
    # print(property_info)
    info = soup.find('script', text=re.compile(r'window\.SAVED_STORE_STATE\s*=\s*{.*}'))
    cleaned_js_object_str = info.string.strip()[len('window.SAVED_STORE_STATE = '):]

closing_brace_index = cleaned_js_object_str.rfind('}')
cleaned_js_object_str = cleaned_js_object_str[:closing_brace_index + 1]
json_object = json.loads(cleaned_js_object_str)
data = pd.DataFrame(json_object["search"]["data"]["properties"]["apartments"])

In [2]:
# # phase 2

# -- Table1 Queries
# -- 1) Order records by rental price in ascending order
# SELECT * FROM Table1 ORDER BY Rental_Price ASC;

# -- 2) Select unique combinations of City and State with their average Rental Price
# SELECT City, State, AVG(Rental_Price) AS Average_Rental_Price 
# FROM Table1 
# GROUP BY City, State;

# -- 3) Select the top 5 highest deposit amounts with corresponding Address and City
# SELECT Address, City, Deposit 
# FROM Table1 
# ORDER BY Deposit DESC 
# LIMIT 5;

# -- 4) Select the count of records for each Country along with the total deposit amount
# SELECT Country, COUNT(*) AS Record_Count, SUM(Deposit) AS Total_Deposit 
# FROM Table1 
# GROUP BY Country;

# -- 5) Select records with a Rental Price higher than the average Rental Price across all records
# SELECT * FROM Table1 WHERE Rental_Price > (SELECT AVG(Rental_Price) FROM Table1);

# -- Table2 Queries
# -- 1) Select the average area for each number of bedrooms
# SELECT Bedrooms, AVG(Area) AS Average_Area 
# FROM Table2 
# GROUP BY Bedrooms;

# -- 2) Select records with more than one bathroom and pets allowed
# SELECT * FROM Table2 WHERE Bathrooms > 1 AND Pets_Allowed = 'Yes';

# -- 3) Select the top 3 records with the highest total area (bedrooms + bathrooms)
# SELECT *, (Bedrooms + Bathrooms) AS Total_Area 
# FROM Table2 
# ORDER BY Total_Area DESC 
# LIMIT 3;

# -- 4) Select the count of records for each combination of bedrooms and bathrooms
# SELECT Bedrooms, Bathrooms, COUNT(*) AS Record_Count 
# FROM Table2 
# GROUP BY Bedrooms, Bathrooms;

# -- 5) Select records with the largest area where pets are allowed
# SELECT * 
# FROM Table2 
# WHERE Pets_Allowed = 'Yes' 
# ORDER BY Area DESC 
# LIMIT 1;

# -- Table3 Queries
# -- 1) Select records where both Washer/Dryer and AC are available, and order by Sno
# SELECT * 
# FROM Table3 
# WHERE Washer_Dryer = 'Available' AND AC = 'Available' 
# ORDER BY Sno;

# -- 2) Select records where Hardwood floors are available but neither Roofdeck nor Storage is present, and order by Sno in descending order
# SELECT * 
# FROM Table3 
# WHERE Hardwood_Floors = 'Available' AND Roofdeck = 'Not Present' AND Storage = 'Not Present' 
# ORDER BY Sno DESC;

# -- 3) Select records where at least four amenities (AC, Parking, Dishwasher, Fireplace) are available, and order by Sno
# SELECT * 
# FROM Table3 
# WHERE AC = 'Available' AND Parking = 'Available' AND Dishwasher = 'Available' AND Fireplace = 'Available' 
# ORDER BY Sno;

# -- 4) Select records where neither Roofdeck nor Storage is available, and include the count of such records
# SELECT *, COUNT(*) AS Record_Count 
# FROM Table3 
# WHERE Roofdeck = 'Not Present' AND Storage = 'Not Present';

# -- 5) Select records with Parking and either Fireplace or Dishwasher, and include the count of records for each condition
# SELECT *, 
#     CASE 
#         WHEN Fireplace = 'Available' THEN 'Parking and Fireplace' 
#         WHEN Dishwasher = 'Available' THEN 'Parking and Dishwasher' 
#     END AS Condition 
# FROM Table3 
# WHERE Parking = 'Available' 
# ORDER BY Condition;

# -- Join SQL Queries using all 3 tables
# -- 1) Subquery to find records with more than the average area and related details using table 1 and table 2
# SELECT * 
# FROM Table1 
# WHERE ID IN (SELECT ID FROM Table2 WHERE Area > (SELECT AVG(Area) FROM Table2));

# -- 2) Subquery to find records in table1 based on conditions pets allowed is ‘YES’ and no of bed is greater than 3 in table2
# SELECT * 
# FROM Table1 
# WHERE ID IN (SELECT ID FROM Table2 WHERE Pets_Allowed = 'Yes' AND Bedrooms > 3);

# -- 3) Subquery using both tables (2 and 3) to find records in Table2 with more than 2 bedrooms and related details from Table3 where AC is present
# SELECT * 
# FROM Table2 
# WHERE Bedrooms > 2 
# AND ID IN (SELECT ID FROM Table3 WHERE AC = 'Available');

# -- 4) SQL subquery to find records in Table2 with pets allowed and a Dishwasher, and include related details from Table3
# SELECT * 
# FROM Table2 
# WHERE Pets_Allowed = 'Yes' 
# AND ID IN (SELECT ID FROM Table3 WHERE Dishwasher = 'Available');

# -- 5) Subquery to find records in Table2 with the highest area and related details from Table3 where roofdeck is present
# SELECT * 
# FROM Table2 
# WHERE Area = (SELECT MAX(Area) FROM Table2) 
# AND ID IN (SELECT ID FROM Table3 WHERE Roofdeck = 'Present');

# -- 6) Inner Join to combine information from table1 and table 2
# SELECT * 
# FROM Table1 
# INNER JOIN Table2 
# ON Table1.ID = Table2.ID;

# -- 7) Subquery to find records in table1 with pets allowed and a Washer/Dryer, and include details from table2 and table3
# SELECT * 
# FROM Table1 
# WHERE ID IN (SELECT ID FROM Table2 WHERE Pets_Allowed = 'Yes' AND Washer_Dryer = 'Available');


In [ ]:
# pahse 3
import pandas as pd

# Read the CSV files into pandas dataframes
df1 = pd.read_csv('file1.csv')
df2 = pd.read_csv('file2.csv')
df3 = pd.read_csv('file3.csv')

# Merge the dataframes using concat
combined_df = pd.concat([df1, df2, df3], ignore_index=True)

# Or you can merge them using merge if there are common columns to join on
# combined_df = pd.merge(df1, df2, on='common_column')
# combined_df = pd.merge(combined_df, df3, on='common_column')

# Perform EDA on the combined dataframe
# For example:
# Display the first few rows of the dataframe
print(combined_df.head())

# Get summary statistics
print(combined_df.describe())

# Check for missing values
print(combined_df.isnull().sum())

# Check data types of columns
print(combined_df.dtypes)

# Perform visualizations (histograms, scatter plots, etc.)
# For example:
import matplotlib.pyplot as plt

# Histogram of Rental Price
plt.hist(combined_df['Rental_Price'], bins=20)
plt.xlabel('Rental Price')
plt.ylabel('Frequency')
plt.title('Distribution of Rental Price')
plt.show()

# Scatter plot of Rental Price vs. Area
plt.scatter(combined_df['Area'], combined_df['Rental_Price'])
plt.xlabel('Area')
plt.ylabel('Rental Price')
plt.title('Rental Price vs. Area')
plt.show()

# Continue exploring the data and creating visualizations as needed

# Write detailed observations of each analysis
# For example:
# Observation 1: 
# The distribution of rental prices is right-skewed, with a few properties having significantly higher rental prices than the majority.
# Observation 2:
# There seems to be a positive correlation between the area of the property and its rental price, indicating that larger properties tend to have higher rental prices.
# Observation 3:
# There are some missing values in the dataset, particularly in columns related to property amenities. These missing values may need to be handled appropriately before further analysis.

In [ ]:
# phase 4

# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib

# Step 1: Data Preprocessing
# Load the combined dataset
data = pd.concat([pd.read_csv('data1.csv'), pd.read_csv('data2.csv'), pd.read_csv('data3.csv')])

# Handle missing values, feature engineering, etc.

# Split the data into features and target variables
X_rental = data.drop(['rental_price'], axis=1)
y_rental = data['rental_price']
X_area = data.drop(['area'], axis=1)
y_area = data['area']

# Step 2: Data Scaling
scaler = StandardScaler()
X_rental_scaled = scaler.fit_transform(X_rental)
X_area_scaled = scaler.fit_transform(X_area)

# Step 3: Train-Test Split
X_rental_train, X_rental_test, y_rental_train, y_rental_test = train_test_split(X_rental_scaled, y_rental, test_size=0.2, random_state=42)
X_area_train, X_area_test, y_area_train, y_area_test = train_test_split(X_area_scaled, y_area, test_size=0.2, random_state=42)

# Step 4: Model Selection and Training
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest Regressor': RandomForestRegressor(),
    'Gradient Boosting Regressor': GradientBoostingRegressor()
}

best_models = {}

for label, model in models.items():
    model.fit(X_rental_train, y_rental_train)
    best_models[label] = model

# Step 5: Hyperparameter Tuning (Optional)
# You can perform hyperparameter tuning using GridSearchCV or RandomizedSearchCV

# Step 6: Model Evaluation
results = {}
for label, model in best_models.items():
    y_pred = model.predict(X_rental_test)
    mae = mean_absolute_error(y_rental_test, y_pred)
    mse = mean_squared_error(y_rental_test, y_pred)
    rmse = mean_squared_error(y_rental_test, y_pred, squared=False)
    results[label] = {'MAE': mae, 'MSE': mse, 'RMSE': rmse}

# Step 7: Select the Best Model
best_model_label = min(results, key=lambda x: results[x]['RMSE'])
best_model = best_models[best_model_label]

# Step 8: Save the Best Model
joblib.dump(best_model, 'best_rental_price_model.pkl')

# Repeat Steps 4-8 for predicting the area

# Final Evaluation
print("Best Model for Rental Price Prediction:", best_model_label)
print("Evaluation Results:")
for label, metrics in results.items():
    print(label)
    print(metrics)
